In [ ]:
import jax
import jax.numpy as np
import numpy as onp
from jax import random, jit, vmap, grad
from jax.experimental import optimizers,stax
from jax.experimental.stax import Dense, Conv, Relu, MaxPool, Flatten, Softmax,BatchNorm,Dropout
from jax.experimental.optimizers import optimizer, make_schedule, exponential_decay
#from jax.ops import index, index_update

import itertools
from functools import partial
from tqdm import trange
from torch.utils import data
import matplotlib.pyplot as plt

from scipy.integrate import odeint

from jax.nn import sigmoid,relu,log_sigmoid,one_hot
from jax.lax import scan
import h5py
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm/ np.sum(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2%' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from keras.datasets import cifar10
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
class DataGenerator(data.Dataset):
    def __init__(self, images, labels, 
                 batch_size=128, 
                 rng_key=random.PRNGKey(1234)):
        'Initialization'
        self.images = images
        self.labels = labels
        self.N = labels.shape[0]
        self.batch_size = batch_size
        self.key = rng_key

    @partial(jit, static_argnums=(0,))
    def __data_generation(self, key, images, labels):
        'Generates data containing batch_size samples'
        idx = random.choice(key, self.N, (self.batch_size,), replace=False)
        images = images[idx,...]
        labels = labels[idx,...]
        return images, labels

    def __getitem__(self, index):
        'Generate one batch of data'
        self.key, subkey = random.split(self.key)
        images, labels = self.__data_generation(self.key, self.images, self.labels)
        return images, labels

In [ ]:
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)

In [ ]:
x_train=x_train/255.
num_classes=y_train.max()+1
y_train=y_train.flatten()
y_train=one_hot(y_train,num_classes)

print(x_train.shape)
print(y_train.shape)

train_dataset = DataGenerator(x_train, y_train, batch_size=128)

(50000, 32, 32, 3)
(50000, 10)


In [ ]:
# Architecture
def CNN():
    init, apply = stax.serial(Conv(6, (5, 5), (1, 1), padding="SAME"),
                              Relu,Dropout(0.85,mode='train'), MaxPool((2, 2), (2, 2)),
                              Conv(12, (5, 5), (1, 1), padding="SAME"),
                              Relu,Dropout(0.85,mode='train'), MaxPool((2, 2), (2, 2)),
                              Conv(24, (5, 5), (1, 1), padding="SAME"),
                              Relu,Dropout(0.85,mode='train'), MaxPool((2, 2), (2, 2)),
                              Flatten,Dropout(0.85,mode='train'),
                              Dense(120), Relu, Dropout(0.85,mode='train'),
                              Dense(84), Relu, Dropout(0.85,mode='train'),
                              Dense(10), Softmax)
    return init, apply

In [ ]:
# Architecture
def CNN_test():
    init, apply = stax.serial(Conv(6, (5, 5), (1, 1), padding="SAME"),
                              Relu,Dropout(0.85,mode='test'), MaxPool((2, 2), (2, 2)),
                              Conv(12, (5, 5), (1, 1), padding="SAME"),
                              Relu,Dropout(0.85,mode='test'), MaxPool((2, 2), (2, 2)),
                              Conv(24, (5, 5), (1, 1), padding="SAME"),
                              Relu,Dropout(0.85,mode='test'), MaxPool((2, 2), (2, 2)),
                              Flatten,Dropout(0.85,mode='test'),
                              Dense(120), Relu,Dropout(0.85,mode='test'),
                              Dense(84), Relu,Dropout(0.85,mode='test'),
                              Dense(10), Softmax)
    return init, apply

In [ ]:
class CNNclassifier:
    # Initialize the class
    def __init__(self, rng_key=random.PRNGKey(0)):
        # MLP init and apply functions
        self.net_init, self.net_apply = CNN()
        self.test_init, self.test_apply = CNN_test()
        _, params = self.net_init(rng_key, (128,32,32,3))

        # Optimizer initialization and update functions
        lr = optimizers.exponential_decay(1e-3, decay_steps=1000, decay_rate=0.999)
        self.opt_init, \
        self.opt_update, \
        self.get_params = optimizers.adam(lr)
        self.opt_state = self.opt_init(params)

        # Logger
        self.itercount = itertools.count()
        # for training
        self.loss_log = []
        self.acc_log = []
        # for testing
        self.loss_log2 = []
        self.acc_log2 = []

    @partial(jit, static_argnums=(0,))
    def accuracy(self, params, batch):
        images, labels = batch
        outputs = self.predict(params, images)
        pred_class = np.argmax(outputs,1)
        true_class = np.argmax(labels,1)
        return np.sum((pred_class == true_class)) / images.shape[0]

    def loss(self, params, batch,it):
        images, labels = batch
        outputs = self.net_apply(params, images, rng=random.PRNGKey(it))
        loss = -labels*np.log(outputs+1e-7)
        return np.mean(loss)

    @partial(jit, static_argnums=(0,))
    def step(self, i, opt_state, batch):
        params = self.get_params(opt_state)
        gradients = grad(self.loss)(params, batch, i)
        return self.opt_update(i, gradients, opt_state)

    def train(self, train_dataset, test_dataset, nIter = 10):
        data = iter(train_dataset)
        data_test = iter(test_dataset)

        pbar = trange(nIter)
        # Main training loop
        for it in pbar:
            # Run one gradient descent update
            batch = next(data)
            self.opt_state = self.step(next(self.itercount), self.opt_state, batch)  
            if it % 50 == 0:
                batch_test = next(data_test)

                # Logger
                params = self.get_params(self.opt_state)

                loss = self.loss(params, batch, it)
                acc = self.accuracy(params, batch)
                self.loss_log.append(loss)
                self.acc_log.append(acc)

                loss2 = self.loss(params, batch_test, it)
                acc2 = self.accuracy(params, batch_test)
                self.loss_log2.append(loss2)
                self.acc_log2.append(acc2)

                pbar.set_postfix({'Loss': loss,
                                  'Loss(test)': loss2,
                                  'Accuracy': acc,
                                  'Accuracy(test)': acc2})

    @partial(jit, static_argnums=(0,))
    def predict(self, params, inputs):
        #outputs = self.net_apply(params, inputs,rng=random.PRNGKey(0))
        outputs = self.test_apply(params, inputs,rng=random.PRNGKey(0))
        return outputs

In [ ]:
# Initialize model
model = CNNclassifier()

KeyboardInterrupt: ignored

In [ ]:
x_test=x_test/255.
num_classes=y_test.max()+1
y_test=y_test.flatten()
y_test=one_hot(y_test,num_classes)
x_test=np.asarray(x_test)
test_dataset = DataGenerator(x_test, y_test, batch_size=128)

In [ ]:
# train model
model.train(train_dataset, test_dataset, nIter=20000)
opt_params = model.get_params(model.opt_state)

In [ ]:
# save outputs
onp.save('dropout_log_loss', model.loss_log)
onp.save('dropout_test_log_loss', model.loss_log2)

In [ ]:
# calculate the number of parameters
def num_params(params):
  num_params = 0
  for i in range(len(params)):
    tup = params[i]
    tup_len = 0
    for j in range(len(tup)):
      tup_j_len = len(params[i][j].ravel())
      tup_len += tup_j_len
    num_params += tup_len
  return num_params

In [ ]:
# num params
print("Number of parameters: ", num_params(opt_params))

In [ ]:
plt.figure(dpi = 150)
plt.xlabel(r'Number of Iterations ($\times 50$)')
plt.ylabel('Accuracy')
plt.semilogy(model.acc_log, label = 'Train')
plt.semilogy(model.acc_log2, label = 'Test')
plt.legend()

In [ ]:
plt.figure(dpi = 150)
plt.xlabel(r'Number of Iterations ($\times 50$)')
plt.ylabel('Cross Entropy Loss')
plt.semilogy(model.loss_log, label = 'Train')
plt.semilogy(model.loss_log2, label = 'Test')
plt.legend()

In [ ]:
test_init, test_apply = CNN_test()

In [ ]:
# accuracy
def predict(params, inputs):
  outputs = test_apply(params, inputs,rng=random.PRNGKey(0))
  # outputs = test_apply(params, inputs)
  return outputs

In [ ]:
# Compute classification accuracy on the entire test data-set
@jit
def accuracy(params, batch):
    images, labels = batch
    outputs = predict(params, images)
    pred_class = np.argmax(outputs,1)
    true_class = np.argmax(labels,1)
    return np.sum((pred_class == true_class)) / images.shape[0]

In [ ]:
acc = accuracy(opt_params, (x_test, y_test))
print('Classification accuracy: {}%'.format(100*acc))

In [ ]:
# save accuracies
onp.save('dropout_acc', model.acc_log)
onp.save('dropout_test_acc', model.acc_log2)